In [6]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class LIDCDataset(Dataset):

    def __init__(self, img_dir, mask_dir):

        self.image_dir = img_dir
        self.mask_dir = mask_dir
        self.image_files = sorted(os.listdir(self.image_dir))
        self.mask_files = sorted(os.listdir(self.mask_dir))

    def __len__(self):

        return len(self.image_files)

    def __getitem__(self, idx):
        
        # Load the numpy arrays
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        image = np.load(img_path).astype(np.float32)
        mask = np.load(mask_path).astype(np.float32)

        # Normalize image and mask between 0 and 1
        image = (image - image.min()) / (image.max() - image.min() + 1e-8)
        mask = (mask - mask.min()) / (mask.max() - mask.min() + 1e-8)

        # Add channel dimension (1, H, W)
        image = np.expand_dims(image, axis=0)
        mask = np.expand_dims(mask, axis=0)

        # Convert to torch tensors
        image = torch.from_numpy(image)
        mask = torch.from_numpy(mask)

        return image, mask


#constructing the dataset
img_dir = "/Users/daniel/Documents/CSAI/Internship/CODE/data/LUNA16/images/"
mask_dir = "/Users/daniel/Documents/CSAI/Internship/CODE/data/LUNA16/labels/"
dataset = LIDCDataset(img_dir, mask_dir)

#splitting the dataset
train_set, val_set, test_set = random_split(dataset, [0.7, 0.15, 0.15])

#data loaders
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

#test
images, masks = next(iter(train_loader))
print("Image batch shape:", images.shape)
print("Mask batch shape:", masks.shape)


Image batch shape: torch.Size([32, 1, 64, 64])
Mask batch shape: torch.Size([32, 1, 64, 64])


In [7]:
#model

import torch
import torch.nn as nn
import torch.nn.functional as F


#variables
conv_kernel_size = 3
conv_padding = 1
upconv_kernel_size = 2
upconv_stride = 2
pool_kernel_size = 2
pool_stride = 2


class Unet(nn.Module):
    def __init__(self):
        super().__init__()

        #encoder
        self.conv1_1 = nn.Conv2d(1, 64, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=conv_kernel_size, padding=conv_padding)

        self.pool = nn.MaxPool2d(kernel_size=pool_kernel_size, stride=pool_stride)

        self.drop1 = nn.Dropout2d(0.1)
        self.drop2 = nn.Dropout2d(0.1)
        self.drop3 = nn.Dropout2d(0.1)
        self.drop4 = nn.Dropout2d(0.1)
        self.drop5 = nn.Dropout2d(0.1)
        self.drop6 = nn.Dropout2d(0.1)
        self.drop7 = nn.Dropout2d(0.1)
        self.drop8 = nn.Dropout2d(0.1)
        self.drop9 = nn.Dropout2d(0.1)

        #bottleneck
        self.conv5_1 = nn.Conv2d(512, 1024, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv5_2 = nn.Conv2d(1024, 1024, kernel_size=conv_kernel_size, padding=conv_padding)

        #decoder
        self.conv6_1 = nn.Conv2d(1024, 512, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv6_2 = nn.Conv2d(512, 512, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv7_1 = nn.Conv2d(512, 256, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv7_2 = nn.Conv2d(256, 256, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv8_1 = nn.Conv2d(256, 128, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv8_2 = nn.Conv2d(128, 128, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv9_1 = nn.Conv2d(128, 64, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv9_2 = nn.Conv2d(64, 64, kernel_size=conv_kernel_size, padding=conv_padding)
        self.conv10 = nn.Conv2d(64, 1, kernel_size=1)

        self.upconv6 = nn.ConvTranspose2d(1024, 512, kernel_size=upconv_kernel_size, stride=upconv_stride)
        self.upconv7 = nn.ConvTranspose2d(512, 256, kernel_size=upconv_kernel_size, stride=upconv_stride)
        self.upconv8 = nn.ConvTranspose2d(256, 128, kernel_size=upconv_kernel_size, stride=upconv_stride)
        self.upconv9 = nn.ConvTranspose2d(128, 64, kernel_size=upconv_kernel_size, stride=upconv_stride)


    def forward(self, x):

        #encoder
        c1 = F.relu(self.conv1_1(x))
        c1 = self.drop1(c1)
        c1 = F.relu(self.conv1_2(c1))
        p1 = self.pool(c1)
        c2 = F.relu(self.conv2_1(p1))
        c2 = self.drop2(c2)
        c2 = F.relu(self.conv2_2(c2))
        p2 = self.pool(c2)  
        c3 = F.relu(self.conv3_1(p2))
        c3 = self.drop3(c3)
        c3 = F.relu(self.conv3_2(c3))
        p3 = self.pool(c3)  
        c4 = F.relu(self.conv4_1(p3))
        c4 = self.drop4(c4)
        c4 = F.relu(self.conv4_2(c4))
        p4 = self.pool(c4)

        #bottleneck
        c5 = F.relu(self.conv5_1(p4))
        c5 = self.drop5(c5)
        c5 = F.relu(self.conv5_2(c5))

        #decoder
        u6 = torch.cat((self.upconv6(c5), c4), dim=1)
        c6 = F.relu(self.conv6_1(u6))
        c6 = self.drop6(c6)
        c6 = F.relu(self.conv6_2(c6))
        u7 = torch.cat((self.upconv7(c6), c3), dim=1)
        c7 = F.relu(self.conv7_1(u7))
        c7 = self.drop7(c7)
        c7 = F.relu(self.conv7_2(c7))
        u8 = torch.cat((self.upconv8(c7), c2), dim=1)
        c8 = F.relu(self.conv8_1(u8))
        c8 = self.drop8(c8)
        c8 = F.relu(self.conv8_2(c8))
        u9 = torch.cat((self.upconv9(c8), c1), dim=1)
        c9 = F.relu(self.conv9_1(u9))
        c9 = self.drop9(c9)
        c9 = F.relu(self.conv9_2(c9))
        output = torch.sigmoid(self.conv10(c9))
        
        return output

In [8]:
#training and validation loop

import torch
import torch.nn as nn
import torch.optim as optim


model = Unet()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5


#training loop
for epoch in range(num_epochs):

    #training
    train_loss = 0.0
    model.train()

    for images, masks in train_loader:

        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        #balance the loss per batch size
        train_loss += loss.item() * images.size(0)
    
    train_loss /= len(train_loader.dataset)


    #validation
    val_loss = 0.0
    model.eval()
    
    with torch.no_grad():

        for images, masks in val_loader:

            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)
            
            #balance the loss per batch size
            val_loss += loss.item() * images.size(0)
        
    val_loss /= len(val_loader.dataset)


    print(f"Epoch [{epoch+1}/{num_epochs}]  Train Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}")

Epoch [1/5]  Train Loss: 0.6820  Val Loss: 0.5548
Epoch [2/5]  Train Loss: 0.4836  Val Loss: 0.3834
Epoch [3/5]  Train Loss: 0.3819  Val Loss: 0.3428
Epoch [4/5]  Train Loss: 0.3503  Val Loss: 0.2971
Epoch [5/5]  Train Loss: 0.3044  Val Loss: 0.2719


In [9]:
#testing loop

def dice_coeff(outputs, masks, smooth=1e-6):

    outputs = outputs.view(-1)
    masks = masks.view(-1)

    intersection = (outputs * masks).sum()
    dice = (2. * intersection + smooth) / (outputs.sum() + masks.sum() + smooth)

    return dice.item()


test_dice = 0.0
test_loss = 0.0
model.eval()

with torch.no_grad():

    for images, masks in test_loader:

        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        dice = dice_coeff(outputs, masks)
        loss = criterion(outputs, masks)
        
        #balance the loss per batch size
        test_dice += dice * images.size(0)
        test_loss += loss.item() * images.size(0)
    
test_dice /= len(test_loader.dataset)    
test_loss /= len(test_loader.dataset)

print(f"Mean dice coefficient on test set: {test_dice:.4f}")
print(f"Mean loss on test set: {test_loss:.4f}")

Mean dice coefficient on test set: 0.1180
Mean loss on test set: 0.2832
